In [ ]:
import folium
from folium import IFrame, PolyLine, Marker
from branca.element import MacroElement, Template
import base64
import os
import pandas as pd

# File: game_ui.py
import pygame
import pygame_widgets
from pygame_widgets.button import ButtonArray
import sys
import random


In [63]:
class Marker:
    def __init__(self, zoom_start=13):
        self.map = folium.Map(location=[0, 0], zoom_start=zoom_start)

    def draw_guess_marker(self):
        map_var = self.map.get_name()
        icon_url = "https://cdn-icons-png.flaticon.com/512/684/684908.png"

        tpl = Template(f"""
        {{% macro script(this, kwargs) %}}
          // define your custom icon once
          var guessIcon = L.icon({{
            iconUrl: "{icon_url}",
            iconSize: [25, 41],
            iconAnchor: [12, 41],
            popupAnchor: [1, -34]
          }});

          var guessMarker;
          {map_var}.on('click', function(e) {{
            if (guessMarker) {{
              // move existing marker and re-apply icon
              guessMarker.setLatLng(e.latlng)
                         .setIcon(guessIcon)
                         .openPopup();
            }} else {{
              // first click: create marker with that icon
              guessMarker = L.marker(e.latlng, {{icon: guessIcon}})
                               .addTo({map_var})
                               .bindPopup("Your Guess:")
                               .openPopup();
            }}
          }});
        {{% endmacro %}}
        """)

        click_control = MacroElement()
        click_control._template = tpl
        guess_marker= self.map.add_child(click_control)
        return guess_marker
    
    def draw_correct_marker(self, csv_path, image_dir):
        image_index = random.randint(0, 2)

        # Read lines from the CSV
        with open(csv_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Check that the index exists
        if image_index < 0 or image_index > len(lines):
            raise IndexError("That image number is out of range.")

        # Extract lat/lon from the line
        line = lines[image_index].strip()
        lat_str, lon_str = line.split(",")
        lat = float(lat_str)
        lon = float(lon_str)

        # Get the corresponding image path
        image_filename = f"{image_index}.jpg"
        image_path = os.path.join(image_dir, image_filename)

        # Use image directly (assuming it's hosted or local)
        if os.path.exists(image_path):
            html = f'<img src="{image_path}" width="300">'
        else:
            html = f"<p>Image not found: {image_filename}</p>"

        iframe = IFrame(html, width=320, height=240)
        popup = folium.Popup(iframe, max_width=320)

        # Create map and add marker
        # why would we create a map if draw_correct alr made one m = folium.Map(location=[lat, lon], zoom_start=self.zoom_start)
        # self.map = folium.Map(location=[lat, lon], zoom_start=self.zoom_start)
        correct_marker= folium.Marker(
            location=[lat, lon],
            popup=popup,
            icon=folium.Icon(color="red", icon="pushpin"),
        ).add_to(self.map)
        
        return correct_marker
        
        # folium.Marker(
        #     location=[lat, lon],
        #     popup=popup,
        #     icon=folium.Icon(color="red", icon="pushpin"),
        # ).add_to(m)
        
        # return m
    
    def line_coords(self, guess_marker, correct_marker):
        return guess_marker, correct_marker

In [ ]:
class Map_Extras: 
    def __init__(self, zoom_start):
        self.zoom_start = zoom_start
        self.map = Marker()
        self.guess_marker, self.correct_marker= Marker.line_coords()
        self.confirm_clicked= False
        self.buttons = Button(
            confirm_callback=self.handle_confirm_guess,
            next_round_callback=self.handle_next_round
        )

    def handle_confirm_guess(self):
        """Called when Confirm Guess is clicked"""
        self.draw_line()
        self.map.draw_correct_marker()  # Assuming this is the correct way to call it
    
    def handle_next_round(self):
        """Called when Next Round is clicked"""
        pass

    def draw_line (self):
        coords_line = [self.guess_marker, self.correct_marker]
        folium.Polyline(
            coords_line,
            color="blue",
            weight="10",
            opacity=0.8
    ).add_to(self.map) 
        
        return self.map
    
    # def confirm_guess_button(self):
    #     # when (1) confirm guess is displayed in terminal 
    #     # then the line and correct_marker should pop up, this 
    #     # is from the pygame window thingies that irene made
        
    #     if "1" in self.get_terminal_output():
    #         self.draw_line()
    #         self.Marker.draw_correct_marker()
    #         self.confirm_clicked=False
    #     return self.map
    
    # def next_round_button(self):
    #     if "2" in self.get_terminal_output():
    #         self.draw_line()
    #         self.Marker.draw_correct_marker()
    #         self.confirm_clicked=False
    #     return self.map


In [56]:
game = Marker(zoom_start=1)
game.draw_guess_marker()
game.draw_correct_marker("funny_dataset/coord.csv", "funny_dataset/images/")

In [ ]:
geo = Marker(zoom_start=2)
map_object= Marker(m.draw_guess_marker)
# map_object = geo.draw_correct_marker("funny_dataset/coord.csv", "funny_dataset/images/")
map_object

AttributeError: 'Map' object has no attribute 'draw_guess_marker'

In [23]:
geo = Map_Extras(zoom_start=13)
maps = geo.draw_line
print(maps)

<bound method Map_Extras.draw_line of <__main__.Map_Extras object at 0x7f6a13b8df70>>


In [4]:
m = folium.Map([-27.5717, -48.6256], zoom_start=10)
m.add_child(MeasureControl())
m

In [ ]:
geo = Marker(zoom_start=13)
map_object = geo.draw_correct_marker("funny_dataset/coord.csv", "funny_dataset/images/")
map_object